In [1]:
import os
import psutil
import geopandas as gpd
from matrixconverters.read_ptv import ReadPTVMatrix
import xarray as xr
from collections import namedtuple

In [2]:
os.chdir("..")

In [3]:
from params.project_params import ENCODING_CP1252, CRS_EPSG_ID_WGS84
from scripts.functions import get_gdf_mobility_stations, get_gdf_mobility_stations_with_npvm_zone, get_skim

In [4]:
path_to_npvm_zones_shp = os.path.join('data', 'Verkehrszonen_Schweiz_NPVM_2017_shp.zip')

In [5]:
path_mobility_stations = os.path.join('data', 'mobility-stationen-und-fahrzeuge-schweiz.csv')

In [6]:
FileNameOldNew = namedtuple("FileNameOldNew", ["file_name_old", "file_name_new"])

In [7]:
skim_path_per_type = {
    'PT_JT': FileNameOldNew('DWV_2017_ÖV_Reisezeit_CH_binär.mtx', 'pt_jt.nc'),
    'PT_NT': FileNameOldNew('144_NTR_(OEV).mtx', 'pt_nt.nc'),
    'PT_Dist': FileNameOldNew('DWV_2017_ÖV_Distanz_CH_binär.mtx', 'pt_dist.nc'),
    'Road_JT': FileNameOldNew('DWV_2017_Strasse_Reisezeit_CH_binär.mtx', 'road_jt.nc'),
    'Road_Dist': FileNameOldNew('DWV_2017_Strasse_Distanz_CH_binär.mtx', 'road_dist.nc')
}

In [8]:
process = psutil.Process()

In [9]:
def print_memory_usage():
    print(process.memory_info().rss / 1024 ** 2)

In [10]:
print_memory_usage()

115.86328125


In [11]:
gdf_npvm_zones = gpd.read_file(path_to_npvm_zones_shp, encoding=ENCODING_CP1252).to_crs(CRS_EPSG_ID_WGS84)
print_memory_usage()

205.046875


In [12]:
gdf_mobility_stations = get_gdf_mobility_stations(path_mobility_stations)
print_memory_usage()

209.40625


In [13]:
gdf_mobility_stations_with_npvm_zone = get_gdf_mobility_stations_with_npvm_zone(gdf_mobility_stations, gdf_npvm_zones)
print_memory_usage()

213.26171875


In [14]:
npvm_zones_with_mobility_station = list(set(gdf_mobility_stations_with_npvm_zone['ID'].values))

In [15]:
def read_skim_filter_and_write(ty, path_old, path_new):
    print_memory_usage()
    print(path_old, path_new)
    skim = ReadPTVMatrix(os.path.join('data', path_old))
    if ty.startswith('PT'):
        skim_filtered = skim.sel(destinations=npvm_zones_with_mobility_station)
    elif ty.startswith('Road'):
        skim_filtered = skim.sel(origins=npvm_zones_with_mobility_station)
    skim_filtered.to_netcdf(os.path.join('data', path_new))
    print_memory_usage()

In [16]:
skim_per_type = {}

In [17]:
for ty, (name_old, name_new) in skim_path_per_type.items():
    read_skim_filter_and_write(ty, name_old, name_new)

213.28515625
DWV_2017_ÖV_Reisezeit_CH_binär.mtx pt_jt.nc
786.55859375
221.15625
144_NTR_(OEV).mtx pt_nt.nc
889.78515625
222.72265625
DWV_2017_ÖV_Distanz_CH_binär.mtx pt_dist.nc
788.1484375
222.74609375
DWV_2017_Strasse_Reisezeit_CH_binär.mtx road_jt.nc
788.18359375
222.78125
DWV_2017_Strasse_Distanz_CH_binär.mtx road_dist.nc
788.18359375


In [18]:
print_memory_usage()
skim_filtered_per_type = {}
skims = []
for ty, (_, name_new) in skim_path_per_type.items():
    skim_filtered_per_type[ty] = xr.open_dataset(os.path.join('data', name_new))
print_memory_usage()

222.78125
232.140625


In [19]:
skim_filtered_per_type['PT_JT']

<xarray.Dataset>
Dimensions:       (zone_no: 7978, zone_no2: 7978, origins: 7978,
                   destinations: 1311)
Coordinates:
  * zone_no       (zone_no) int32 101001 201001 201002 ... 710101001 730101001
  * zone_no2      (zone_no2) int32 101001 201001 201002 ... 710101001 730101001
  * origins       (origins) int32 101001 201001 201002 ... 710101001 730101001
  * destinations  (destinations) int32 575701002 15401001 ... 163201010 9601011
Data variables:
    matrix        (origins, destinations) float64 ...
    zone_name     (zone_no) object ...
    zone_names2   (zone_no2) object ...
Attributes:
    fn:                   data\DWV_2017_ÖV_Reisezeit_CH_binär.mtx
    ZeitVon:              nan
    ZeitBis:              nan
    Faktor:               1.0
    VMAktKennung:         0
    AnzBezeichnerlisten:  1
    roundproc:            1
    diagsum:              103179.79940513325

In [20]:
from_zone_id = 223401003
to_zone_id = 352101001

In [57]:
skim_filtered_per_type['Road_Dist'].sel(destinations=to_zone_id).matrix.values

array([379.07564256, 167.90200363, 174.66876891, ..., 193.34894258,
       125.685256  , 186.41989468])

In [40]:
skim_filtered_per_type['PT_NT'].sel(origins=from_zone_id).matrix.values

array([3.03655777, 3.54461811, 3.95955609, ..., 3.6514299 , 3.        ,
       3.65143429])

In [61]:
type(skim_filtered_per_type['PT_JT'].sel(origins=from_zone_id))

xarray.core.dataset.Dataset

In [142]:
%%time
a = skim_filtered_per_type['PT_JT'].sel(origins=from_zone_id).matrix.values
b = skim_filtered_per_type['Road_JT'].sel(destinations=to_zone_id).matrix.values

CPU times: total: 15.6 ms
Wall time: 15.8 ms


In [143]:
a

array([142.3131933 , 161.00422752, 215.75539639, ..., 171.52577216,
       215.91022864, 171.69048687])

In [144]:
b

array([266.61322653, 132.80123485, 138.66285473, ..., 146.64044781,
       109.78028647, 144.99087531])

In [145]:
a + b

array([408.92641983, 293.80546237, 354.41825112, ..., 318.16621997,
       325.69051511, 316.68136217])

In [150]:
def calc_costs(from_zone_id, to_zone_id, skim_pt_jt, skim_pt_nt, skim_pt_dist, skim_road_jt, skim_road_dist, vtts, pt_min_per_transfer, pt_chf_per_km, road_chf_per_km):
    return vtts / 60 * (skim_pt_jt.sel(origins=from_zone_id).matrix.values + pt_min_per_transfer / 60 * skim_pt_nt.sel(origins=from_zone_id).matrix.values + 1 / 60 * skim_road_jt.sel(destinations=to_zone_id).matrix.values) + pt_chf_per_km * skim_pt_dist.sel(origins=from_zone_id).matrix.values + road_chf_per_km * skim_road_dist.sel(destinations=to_zone_id).matrix.values

In [154]:
%%time
for vtts in range(0, 100):
    calc_costs(from_zone_id, to_zone_id, 
               skim_filtered_per_type['PT_JT'], skim_filtered_per_type['PT_NT'], skim_filtered_per_type['PT_Dist'],
               skim_filtered_per_type['Road_JT'], skim_filtered_per_type['Road_Dist'],
               vtts, 20.0, 0.16, 0.75
              )

CPU times: total: 1.3 s
Wall time: 1.31 s


In [32]:
20.0 * skim_filtered_per_type['PT_JT'].sel(origins=from_zone_id) + 15.0 * skim_filtered_per_type['Road_JT'].sel(destinations=to_zone_id)

TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
npvm_zone_per_mobility_station = {}
for e in gdf_mobility_stations_with_npvm_zone[['Stationsnummer', 'ID']].to_dict('records'):
    mob_stat_nr = e['Stationsnummer']
    npvm_id = e['ID']
    if mob_stat_nr in npvm_zone_per_mobility_station:
        if npvm_id != npvm_zone_per_mobility_station[mob_stat_nr]:
            raise ValueError(f'npvm zone for mobility station {mob_stat_nr} not unique')
    else:
        npvm_zone_per_mobility_station[mob_stat_nr] = npvm_id

In [ ]:
%%time
orig_npvm_zone = 201001
data_per_mob_st_id = {}
for mob_st_id, npvm_zone_id in npvm_zone_per_mobility_station.items():
    data = []
    for s in skims:
        data += [s.sel(origins=orig_npvm_zone, destinations=npvm_zone_id)]
    data_per_mob_st_id[mob_st_id, npvm_zone_id] = data
        


In [ ]:
for e in s.sel(origins=orig_npvm_zone):
    print(e.)
    break

In [ ]:
asdf = s.sel(origins=orig_npvm_zone)

In [ ]:
for e in asdf:
    print(e.values)
    break